### Problem Statement


In [ ]:
import sys
import os
import json

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    # Assumes the notebook is in 'labs/Day_01_.../'
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
except IndexError:
    # Fallback for different execution environments
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, clean_llm_output, load_artifact

# Initialize the LLM client. You can change the model here.
# For example: setup_llm_client(model_name="gemini-2.5-flash")
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

In [ ]:
## Generate the PRD

problem_statement = f"""
We want to write a simple application that connects users to vacation rentals. 
The application should allow users to get a recommendation for vacation rentals based on 
their desired activities and/or attractions they wish to visit.

Users will have the ability to make a reservation at their selected property.

"""

prd_prompt = f"""# Generate a Product Requirements Document (PRD) for the following problem statement:
{problem_statement} 
# The PRD should include the following sections:
1. **Problem Statement**: A clear and concise description of the problem.  
2. **Objectives**: The goals we want to achieve with this application.
3. **User Stories**: A list of user stories that describe the features from the user's perspective.
4. **Acceptance Criteria**: The criteria that must be met for the user stories to be considered complete.
5. **Out of Scope**: Features that are not included in the initial release.

 # We are focused on the initial MVP (Minimum Viable Product) and will include only the essential features.
    1. User can specify a list of activities or attractions they want to visit.
    2. User can get a recommendation for vacation rentals based on their desired activities or attractions
    3. User can make a reservation at their selected property.

# The PRD should be in Markdown format and should be well-structured.
  The PRD should be concise and to the point, avoiding unnecessary details.
    
  """  

print("--- Generating  PRD ---")

prd_output = get_completion(prd_prompt, client, model_name, api_provider)
prd_output_clean = clean_llm_output(prd_output, "markdown")
save_artifact( prd_output_clean, "capstone_artifacts/prd.md",)



In [ ]:

# Load the PRD content for schema generation
prd_content = load_artifact("capstone_artifacts/prd.md")
schema_prompt = f"""
You are an expert Database Administrator.
Using this PRD as a reference: {prd_content}
From this PRD, create a normalized SQL schema with only the following three tables:
1. **Users**: To store user information and their interests
2. **Properties**: To store vacation rental properties located in the US.  Include fields for full address.
        No need for latititude/longitude info.
3. **Reservations**: To store user reservations for properties.
The output should be the raw `CREATE TABLE` statements.  SQL should be compatible with sqllite3.
"""

print("--- Generating SQL Schema ---")
if prd_content:
    generated_schema = get_completion(schema_prompt, client, model_name, api_provider)
    
    # Clean up the generated schema using our helper function
    cleaned_schema = clean_llm_output(generated_schema, language='sql')
    print(cleaned_schema)
    
    # Save the cleaned schema
    save_artifact(cleaned_schema, 'capstone_artifacts/schema.sql')
else:
    print("Skipping schema generation because PRD is missing.")
    cleaned_schema = ""